<a href="https://colab.research.google.com/github/Angel-dash/Abstractive-Text-Summarization/blob/main/DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
!unzip /content/drive/MyDrive/MiniProjectdata/data.zip


Archive:  /content/drive/MyDrive/MiniProjectdata/data.zip
  inflating: cnn_dailymail/test.csv  
  inflating: cnn_dailymail/train.csv  
  inflating: cnn_dailymail/validation.csv  


In [3]:
import numpy as np
import pandas as pd
import re
from bs4 import BeautifulSoup
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import warnings
pd.set_option("display.max_colwidth", 200)
warnings.filterwarnings("ignore")
import textwrap
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
data_train=pd.read_csv('/content/cnn_dailymail/train.csv')
data_test=pd.read_csv('/content/cnn_dailymail/test.csv')
data_validation=pd.read_csv('/content/cnn_dailymail/validation.csv')

Data preprocessing

In [5]:
data = pd.concat([data_train, data_test, data_validation ])
df = data

In [6]:
new_df=df.sample(75000)

In [7]:
new_df.head(3)

,id,article,highlights
237847,bfdad69e453b1170d74a81b34787efb3c99a0a53,"By . Ian Drury . PUBLISHED: . 22:35 EST, 11 November 2012 . | . UPDATED: . 02:51 EST, 13 November 2012 . A British Army officer who was shot dead by an Afghan soldier as he played in a Remembrance...","Captain Walter Barrie, 41, died after a rogue soldier hit him during the game .\nWife and fellow soldiers pay tribute to a 'great man' who was 'approachable and compassionate'\nBritish forces hold..."
111408,1ba691af7453d82294d124b0f2d6ac3b85f4e524,"By . Suzannah Hills . PUBLISHED: . 14:05 EST, 1 October 2013 . | . UPDATED: . 05:33 EST, 2 October 2013 . Drug mules Michaella McCollum and Melissa Reid are in brilliant health in a modern well-eq...",Father Maurice Foley visits Michaella McCollum and Melissa Reid at new jail .\nBoth women appeared at private court hearing at Sarita Colonia del Callao .\nThey appeared to have new clothes and Mc...
285878,fe693322187d83dc154628344ca3f9579e6d7c3b,"Washington (CNN) -- On a typical Monday morning, lines of travelers heading toward the security checkpoint at Washington's Reagan National Airport snake past a sign that reads: ""Not much longer. 2...","NEW: Two Air Force bases cancel air shows, citing budget cuts and sequestration .\nLooming cuts in spending could reduce services, impact the military, teachers, border patrols .\nSome Republicans..."


In [8]:
new_df.shape

(75000, 3)

In [9]:
sample=new_df.sample()
print("****Article****")
print(sample.article.values)
print('****Highlights****')
print(sample.highlights.values)

****Article****
['London (CNN) -- With her glossy dark hair, exotic outfit and ruby-red pouting lips, she\'s every inch the classic pin-up... except, that is, for that greenish-blue skin. And yet her image was every bit as popular, in its day, as those of the latest Hollywood starlets. She is the "Chinese Girl," by Vladimir Tretchikoff, the Russian-born South African who became king of the kitsch portrait. In the 1950s and \'60s, no self-respecting suburban home was fit to be seen without a print of her on the living room wall. At one point, the picture was reputed to be the most reproduced image in the world; on Wednesday, the original sold for almost $1.5 million (Â£982,050) at Bonhams auction house in London, far outstripping pre-sale estimates of $750,000. Speaking to CNN before the auction, Giles Peppiatt, director of South African art Bonhams said that the picture\'s kitschy popular appeal "isn\'t necessarily a bad thing." "It is an extraordinary image... and it certainly seems t

Checking for Null values


In [10]:
new_df.isna().sum()

id            0
article       0
highlights    0
dtype: int64

Checking for duplicates


In [11]:
new_df.duplicated(subset=['article','highlights']).sum()

176

In [12]:
new_df=new_df.drop_duplicates(subset=['article','highlights'])
new_df.shape

(74824, 3)

In [13]:
new_df.duplicated(subset=['article','highlights']).sum()

0

In [14]:
new_df['article'][:10]

237847    By . Ian Drury . PUBLISHED: . 22:35 EST, 11 November 2012 . | . UPDATED: . 02:51 EST, 13 November 2012 . A British Army officer who was shot dead by an Afghan soldier as he played in a Remembrance...
111408    By . Suzannah Hills . PUBLISHED: . 14:05 EST, 1 October 2013 . | . UPDATED: . 05:33 EST, 2 October 2013 . Drug mules Michaella McCollum and Melissa Reid are in brilliant health in a modern well-eq...
285878    Washington (CNN) -- On a typical Monday morning, lines of travelers heading toward the security checkpoint at Washington's Reagan National Airport snake past a sign that reads: "Not much longer. 2...
38802     Emilie Livingston may have been launched into the limelight after marrying Jeff Goldblum earlier this month, but the newlywed is also famous in her own right. The 31-year-old from Toronto has quit...
135626    Berlin (CNN) -- Pope Benedict XVI met Friday with a group of people who had been sexually abused by clergy in his native Germany, where disenchantment

In [15]:
new_df['article'] = new_df['article'].str.lower()
new_df['highlights'] = new_df['highlights'].str.lower()

new_df.head()

,id,article,highlights
237847,bfdad69e453b1170d74a81b34787efb3c99a0a53,"by . ian drury . published: . 22:35 est, 11 november 2012 . | . updated: . 02:51 est, 13 november 2012 . a british army officer who was shot dead by an afghan soldier as he played in a remembrance...","captain walter barrie, 41, died after a rogue soldier hit him during the game .\nwife and fellow soldiers pay tribute to a 'great man' who was 'approachable and compassionate'\nbritish forces hold..."
111408,1ba691af7453d82294d124b0f2d6ac3b85f4e524,"by . suzannah hills . published: . 14:05 est, 1 october 2013 . | . updated: . 05:33 est, 2 october 2013 . drug mules michaella mccollum and melissa reid are in brilliant health in a modern well-eq...",father maurice foley visits michaella mccollum and melissa reid at new jail .\nboth women appeared at private court hearing at sarita colonia del callao .\nthey appeared to have new clothes and mc...
285878,fe693322187d83dc154628344ca3f9579e6d7c3b,"washington (cnn) -- on a typical monday morning, lines of travelers heading toward the security checkpoint at washington's reagan national airport snake past a sign that reads: ""not much longer. 2...","new: two air force bases cancel air shows, citing budget cuts and sequestration .\nlooming cuts in spending could reduce services, impact the military, teachers, border patrols .\nsome republicans..."
38802,6dacc8a07a994febb00f50d4e4ebab1b9b0a9e38,"emilie livingston may have been launched into the limelight after marrying jeff goldblum earlier this month, but the newlywed is also famous in her own right. the 31-year-old from toronto has quit...","the 31-year-old from toronto moved to russia at just ten years old to train with renowned rhythmic gymnastics coaches .\nshe went on to become a three-time national champion of canada, as well as ..."
135626,3b791e59d020a5feebfbc0aa9f549c87eb2584ce,"berlin (cnn) -- pope benedict xvi met friday with a group of people who had been sexually abused by clergy in his native germany, where disenchantment with the roman catholic church has grown in t...","pope benedict xvi meets with victims of clergy abuse in his native germany .\nhe ""expressed his deep compassion and regret,"" the vatican reports .\nit is his first state visit to germany, which in..."


In [16]:
new_df['article']=new_df['article'].str.lower()

new_df['highlights']=new_df['highlights'].str.lower()

Converting to lower case

In [17]:
new_df.head()

,id,article,highlights
237847,bfdad69e453b1170d74a81b34787efb3c99a0a53,"by . ian drury . published: . 22:35 est, 11 november 2012 . | . updated: . 02:51 est, 13 november 2012 . a british army officer who was shot dead by an afghan soldier as he played in a remembrance...","captain walter barrie, 41, died after a rogue soldier hit him during the game .\nwife and fellow soldiers pay tribute to a 'great man' who was 'approachable and compassionate'\nbritish forces hold..."
111408,1ba691af7453d82294d124b0f2d6ac3b85f4e524,"by . suzannah hills . published: . 14:05 est, 1 october 2013 . | . updated: . 05:33 est, 2 october 2013 . drug mules michaella mccollum and melissa reid are in brilliant health in a modern well-eq...",father maurice foley visits michaella mccollum and melissa reid at new jail .\nboth women appeared at private court hearing at sarita colonia del callao .\nthey appeared to have new clothes and mc...
285878,fe693322187d83dc154628344ca3f9579e6d7c3b,"washington (cnn) -- on a typical monday morning, lines of travelers heading toward the security checkpoint at washington's reagan national airport snake past a sign that reads: ""not much longer. 2...","new: two air force bases cancel air shows, citing budget cuts and sequestration .\nlooming cuts in spending could reduce services, impact the military, teachers, border patrols .\nsome republicans..."
38802,6dacc8a07a994febb00f50d4e4ebab1b9b0a9e38,"emilie livingston may have been launched into the limelight after marrying jeff goldblum earlier this month, but the newlywed is also famous in her own right. the 31-year-old from toronto has quit...","the 31-year-old from toronto moved to russia at just ten years old to train with renowned rhythmic gymnastics coaches .\nshe went on to become a three-time national champion of canada, as well as ..."
135626,3b791e59d020a5feebfbc0aa9f549c87eb2584ce,"berlin (cnn) -- pope benedict xvi met friday with a group of people who had been sexually abused by clergy in his native germany, where disenchantment with the roman catholic church has grown in t...","pope benedict xvi meets with victims of clergy abuse in his native germany .\nhe ""expressed his deep compassion and regret,"" the vatican reports .\nit is his first state visit to germany, which in..."


In [18]:
#removing html tags
import re
def remove_html_tags(text):
    if isinstance(text, str):
        pattern = re.compile('<.*?>')
        return pattern.sub(r'', text)
    else:
        return text

In [19]:
new_df['article'] = new_df['article'].apply(remove_html_tags)

new_df['highlights'] = new_df['highlights'].apply(remove_html_tags)

In [20]:
new_df.head()

,id,article,highlights
237847,bfdad69e453b1170d74a81b34787efb3c99a0a53,"by . ian drury . published: . 22:35 est, 11 november 2012 . | . updated: . 02:51 est, 13 november 2012 . a british army officer who was shot dead by an afghan soldier as he played in a remembrance...","captain walter barrie, 41, died after a rogue soldier hit him during the game .\nwife and fellow soldiers pay tribute to a 'great man' who was 'approachable and compassionate'\nbritish forces hold..."
111408,1ba691af7453d82294d124b0f2d6ac3b85f4e524,"by . suzannah hills . published: . 14:05 est, 1 october 2013 . | . updated: . 05:33 est, 2 october 2013 . drug mules michaella mccollum and melissa reid are in brilliant health in a modern well-eq...",father maurice foley visits michaella mccollum and melissa reid at new jail .\nboth women appeared at private court hearing at sarita colonia del callao .\nthey appeared to have new clothes and mc...
285878,fe693322187d83dc154628344ca3f9579e6d7c3b,"washington (cnn) -- on a typical monday morning, lines of travelers heading toward the security checkpoint at washington's reagan national airport snake past a sign that reads: ""not much longer. 2...","new: two air force bases cancel air shows, citing budget cuts and sequestration .\nlooming cuts in spending could reduce services, impact the military, teachers, border patrols .\nsome republicans..."
38802,6dacc8a07a994febb00f50d4e4ebab1b9b0a9e38,"emilie livingston may have been launched into the limelight after marrying jeff goldblum earlier this month, but the newlywed is also famous in her own right. the 31-year-old from toronto has quit...","the 31-year-old from toronto moved to russia at just ten years old to train with renowned rhythmic gymnastics coaches .\nshe went on to become a three-time national champion of canada, as well as ..."
135626,3b791e59d020a5feebfbc0aa9f549c87eb2584ce,"berlin (cnn) -- pope benedict xvi met friday with a group of people who had been sexually abused by clergy in his native germany, where disenchantment with the roman catholic church has grown in t...","pope benedict xvi meets with victims of clergy abuse in his native germany .\nhe ""expressed his deep compassion and regret,"" the vatican reports .\nit is his first state visit to germany, which in..."


In [21]:
#removing punchuation


import string
string.punctuation
exclude = string.punctuation
def remove_punc(text):
    if isinstance(text, str):
        return ''.join(ch for ch in text if ch not in exclude)
    else:
        return str(text)  # Convert non-string data to string

In [22]:
new_df['article'] = new_df['article'].apply(remove_punc)

new_df['highlights'] = new_df['highlights'].apply(remove_punc)

In [23]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')

print(stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [24]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
new_df['article'] = new_df['article'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
new_df['highlights'] = new_df['highlights'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
data_train.head(1)

In [26]:
new_df.head(1)

,id,article,highlights
237847,bfdad69e453b1170d74a81b34787efb3c99a0a53,ian drury published 2235 est 11 november 2012 updated 0251 est 13 november 2012 british army officer shot dead afghan soldier played remembrance day football match named ministry defence captain w...,captain walter barrie 41 died rogue soldier hit game wife fellow soldiers pay tribute great man approachable compassionate british forces hold man responsible named mohammad ashraf ashraf shot inj...


In [27]:
#!pip install contractions


In [28]:
max_len_text=80
max_len_summary=10

In [29]:
from sklearn.model_selection import train_test_split
x_tr,x_val,y_tr,y_val=train_test_split(df['article'],df['highlights'],test_size=0.1,random_state=0,shuffle=True)

In [ ]:
#prepare a tokenizer for reviews on training data
x_tokenizer = Tokenizer()
x_tokenizer.fit_on_texts(list(x_tr))

#convert text sequences into integer sequences
x_tr    =   x_tokenizer.texts_to_sequences(x_tr)
x_val   =   x_tokenizer.texts_to_sequences(x_val)

#padding zero upto maximum length
x_tr    =   pad_sequences(x_tr,  maxlen=max_len_text, padding='post')
x_val   =   pad_sequences(x_val, maxlen=max_len_text, padding='post')

x_voc_size   =  len(x_tokenizer.word_index) +1

In [ ]:
#preparing a tokenizer for summary on training data
y_tokenizer = Tokenizer()
y_tokenizer.fit_on_texts(list(y_tr))

#convert summary sequences into integer sequences
y_tr    =   y_tokenizer.texts_to_sequences(y_tr)
y_val   =   y_tokenizer.texts_to_sequences(y_val)

#padding zero upto maximum length
y_tr    =   pad_sequences(y_tr, maxlen=max_len_summary, padding='post')
y_val   =   pad_sequences(y_val, maxlen=max_len_summary, padding='post')

y_voc_size  =   len(y_tokenizer.word_index) +1

1. Note for tomorrow
2. Concatinate all three data set and form a new data set
3. Create a sample of around 100000 of those data
4. Clean those data set and apply all the preprocessing
5. Split into train and validation set
6. Tokenize
7. Then LSTM network